In [3]:

import sys
sys.path.append('/home/nick/work/MLIR/build/tools/mlir/python_packages/mlir_core')
sys.path.append('/home/nick/work/mlir_nelli')

import numpy as np

import mlir.extras.types as T
from mlir.extras.ast.canonicalize import canonicalize
from mlir.extras.context import ContextManager
from mlir.extras.dialects.ext.arith import constant
from mlir.extras.dialects.ext.memref import S
from mlir.extras.dialects.ext.func import toMLIRFunc
from mlir.extras.dialects.ext.scf import canonicalizer as scf_canonicalizer, range_ as range
from mlir.extras.runtime.passes import Pipeline, run_pipeline
from mlir.extras.runtime.refbackend import LLVMJITBackend
from mlir.ir import StridedLayoutAttr

# create MLIR context
CM = ContextManager()
Ctx = CM.__enter__()
backend = LLVMJITBackend()

K = 10
memref_i64 = T.memref(K, K, T.i64())

@toMLIRFunc(emit=True)
@canonicalize(using=scf_canonicalizer) # AST transformer and Bytecode patcher
def memfoo(A: memref_i64, B: memref_i64, C: memref_i64):
    one = constant(1)
    two = constant(2)
    if one > two:
        C[0, 0] = constant(3, T.i64())
    else:
        for i in range(0, K):
            for j in range(0, K):
                C[i, j] = A[i, j] * B[i, j]


PP=Pipeline()
print(str(PP))


module = backend.compile(
    Ctx.module,
    kernel_name=memfoo.__name__,
    pipeline=Pipeline().cse().bufferize().lower_to_llvm(),
)
print(module)
CM.__exit__(None, None, None)


module {
  llvm.func @memfoo(%arg0: !llvm.ptr, %arg1: !llvm.ptr, %arg2: i64, %arg3: i64, %arg4: i64, %arg5: i64, %arg6: i64, %arg7: !llvm.ptr, %arg8: !llvm.ptr, %arg9: i64, %arg10: i64, %arg11: i64, %arg12: i64, %arg13: i64, %arg14: !llvm.ptr, %arg15: !llvm.ptr, %arg16: i64, %arg17: i64, %arg18: i64, %arg19: i64, %arg20: i64) attributes {llvm.emit_c_interface} {
    %0 = llvm.mlir.constant(false) : i1
    %1 = llvm.mlir.constant(3 : i64) : i64
    %2 = llvm.mlir.constant(0 : index) : i64
    %3 = llvm.mlir.constant(10 : index) : i64
    %4 = llvm.mlir.constant(1 : index) : i64
    llvm.cond_br %0, ^bb1, ^bb2
  ^bb1:  // pred: ^bb0
    %5 = llvm.mul %2, %3  : i64
    %6 = llvm.add %5, %2  : i64
    %7 = llvm.getelementptr %arg15[%6] : (!llvm.ptr, i64) -> !llvm.ptr, i64
    llvm.store %1, %7 : i64, !llvm.ptr
    llvm.br ^bb9
  ^bb2:  // pred: ^bb0
    llvm.br ^bb3(%2 : i64)
  ^bb3(%8: i64):  // 2 preds: ^bb2, ^bb7
    %9 = llvm.icmp "slt" %8, %3 : i64
    llvm.cond_br %9, ^bb4, ^bb8
  ^b

False